# Constantes

In [36]:
CHECKSUM_PATH = "./checksum-xmls.csv"
XSL = "../../helpers/reader/passage.transform.xsl"

# Récupération des éléments du corpus

In [5]:
import glob
from MyCapytain.resolvers.cts.local import CtsCapitainsLocalResolver

repositories = list(glob.glob("../../data/raw/corpora/**/*", recursive=False))
resolver = CtsCapitainsLocalResolver(repositories)

../../data/raw/corpora/lascivaroma_additional-texts/lascivaroma_additional-texts/data/phi1351/phi005/phi1351.phi005.perseus-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng4.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi001/phi0448.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi002/phi0448.phi002.perseus-eng2.xml is 

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi002/phi0690.phi002.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi003/phi0690.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1017/phi011/phi1017.phi011.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0660/phi003/phi0660.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1002/phi001/phi1002.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0914/phi001/phi0914.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0917/phi001/phi0917.phi

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1056/phi001/phi1056.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi004/phi1351.phi004.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi002/phi1351.phi002.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0550/phi001/phi0550.phi001.perseus-eng1.xml is not present


# Génération des fichiers XML

## Nettoyage

In [3]:
!rm ./lemmatized/xml/*.xml
!rm ./lemmatized/*.xml

rm: cannot remove './lemmatized/xml/*.xml': No such file or directory
rm: cannot remove './lemmatized/*.xml': No such file or directory


## Récupération des données dépôts

In [53]:
REPOS = {
    
}
with open("../../data/raw/corpora.csv") as f:
    for lineno, line in enumerate(f):
        line = line.strip().split(";")
        if lineno == 0:
            headers = line
        else:
            line = dict(zip(headers, line))
            REPOS[line["Name"]] = line["Version"]

In [59]:
## Chargement des checksums
from pie_extended.utils.hashcheck import md5sum, check_checksum_from_file, read_checksum_csv
import os.path

former_checksums = read_checksum_csv(CHECKSUM_PATH)
xsl_checksum = md5sum(XSL)

repositories_map = {
    "OpenGreekAndLatin_Latin": "OpenGreekAndLatin/Latin",
    "OpenGreekAndLatin_csel-dev": "OpenGreekAndLatin/csel-dev",
    "lascivaroma_additional-texts": "lascivaroma/additional-texts",
    "lascivaroma_priapeia": "lascivaroma/priapeia",
    "PerseusDL_canonical-latinLit": "PerseusDL/canonical-latinLit",
    "ponteineptique_digiliblt": "ponteineptique/digiliblt"
}
file_to_urn = {
    plaintext.split("/")[-1][:-4]: {
        "plaintext_checksum": info.checksum,
        "source_checksum": info.source_checksum,
        "repo": repositories_map[
            info.source.replace("/home/thibault/dev/these/data/raw/corpora/", "").split("/")[0]
        ],
        "version": REPOS[
            repositories_map[
                info.source.replace("/home/thibault/dev/these/data/raw/corpora/", "").split("/")[0]
            ]
        ],
        
    }
    for plaintext, info in former_checksums.items()
}

## Génération

In [66]:
import regex as re
import os.path
import glob
import tqdm
import lxml.etree as etree



TEI = """<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader n="{tid}">
        <fileDesc>
            <titleStmt>
                <title>{title}</title>
                <author>{author}</author>
            </titleStmt>
            <publicationStmt>
                <publisher>
                    <persName>Thibault Clérice</persName>
                </publisher>
            </publicationStmt>
            <sourceDesc>
                <bibl>
                    <idno>{tid}</idno>
                    <link target="{uri}"/>
                    <dim source="xml" type="md5-checksum">{checksum_source}</dim>
                    <dim source="plaintext-transformation" type="md5-checksum">{checksum_text}</dim>
                    <dim source="xsl" type="md5-checksum">{checksum_xsl}</dim>
                </bibl>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
    <text n="{tid}">
        <body>
            
        </body>
    </text>
</TEI>
"""
reference = re.compile(r"(\[REF:[A-Za-z0-9\.]+\])")
for file in glob.glob("./lemmatized/tsv/*-pie.txt"):
    with open(file) as read:
        xml = file.replace("-pie.txt", ".xml")
        if xml == file:
            print(xml, file)
            break
        text_id = os.path.basename(file)[:-len("-pie.txt")]
        metadata = resolver.getMetadata(text_id)
        title = metadata.parent.get_label()
        author = metadata.parent.parent.get_label()
        version_info = file_to_urn[text_id]
        
        root = etree.fromstring(TEI.format(
                tid=text_id,
                title=title,
                author=author,
                checksum_source=version_info["source_checksum"],
                checksum_text=version_info["plaintext_checksum"],
                checksum_xsl=xsl_checksum,
                uri="https://github.com/{name}/archive/{version}.zip".format(
                    name=version_info["repo"], version=version_info["version"]
                )
            ))
        
        body = root.xpath("//t:body", namespaces={"t": "http://www.tei-c.org/ns/1.0"})[0]
        #with open(xml, "w") as write:
        last_seg = None
        for lineno, line in tqdm.tqdm(enumerate(read)):
            line = line.strip().split("\t")
            if lineno == 0:
                headers = line
                continue
            line = dict(zip(headers, line))


            # Deal with Segments
            if line["token"].startswith("[REF:"):
                section, *sid = line["token"][len("[REF:"):-1].split(".")
                sid = ".".join(sid)
                last_seg = etree.Element("{http://www.tei-c.org/ns/1.0}ab", type=section, n=f"{text_id}:{sid}")
                last_seg.tail = "\n            "
                last_seg.text = "\n                "
                body.append(last_seg)
                w_id = 1
                continue

            elif line["token"] in {"<", ">", "&"}:
                continue
            new_word = etree.Element(
                "{http://www.tei-c.org/ns/1.0}w",
                rend=section,
                n=sid,
                pos=line["pos"],
                msd=line["morph"],
                lemma=line["lemma"],
            )
            new_word.text = line["token"]
            new_word.tail = "\n                "
            last_seg.append(new_word)
            w_id += 1

        with open(xml, "w") as f:
            f.write(etree.tostring(base, encoding=str, pretty_print=True))
            
            

512it [00:00, 97017.56it/s]
5487it [00:00, 97453.98it/s]
469it [00:00, 40699.49it/s]
61991it [00:00, 131261.77it/s]
16189it [00:00, 82388.84it/s]
16488it [00:00, 112016.05it/s]
1646it [00:00, 51692.36it/s]
7974it [00:00, 115423.62it/s]
1080it [00:00, 45461.69it/s]
12249it [00:00, 124887.34it/s]
13290it [00:00, 110094.55it/s]
89349it [00:00, 126269.96it/s]
1478it [00:00, 13707.06it/s]
10256it [00:00, 134078.42it/s]
149125it [00:01, 126972.89it/s]
31690it [00:00, 75947.69it/s]
14200it [00:00, 98658.45it/s]
4281it [00:00, 88509.85it/s]
17778it [00:00, 125506.99it/s]
140800it [00:01, 130273.06it/s]
93143it [00:00, 105344.69it/s]
23230it [00:00, 81108.08it/s]
10700it [00:00, 98055.78it/s]
2663it [00:00, 80228.64it/s]
8114it [00:00, 128614.12it/s]
12841it [00:00, 121812.28it/s]
62207it [00:00, 128171.85it/s]
10146it [00:00, 64103.47it/s]
679it [00:00, 41853.66it/s]
13019it [00:00, 124138.72it/s]
8118it [00:00, 107788.32it/s]
13155it [00:00, 122861.39it/s]
3778it [00:00, 81613.94it/s]
10371it

736it [00:00, 5255.55it/s]
55089it [00:00, 130840.80it/s]
4912it [00:00, 47876.42it/s]
15692it [00:00, 121948.41it/s]
368it [00:00, 19020.61it/s]
43091it [00:00, 133812.57it/s]
18363it [00:00, 95732.56it/s]
4146it [00:00, 80644.36it/s]
17775it [00:00, 128199.24it/s]
25203it [00:00, 116721.35it/s]
87708it [00:00, 119149.98it/s]
7153it [00:00, 46655.27it/s]
6405it [00:00, 110442.67it/s]
5033it [00:00, 113056.62it/s]
6487it [00:00, 121757.64it/s]
4493it [00:00, 110962.64it/s]
18113it [00:00, 129514.56it/s]
4379it [00:00, 83697.62it/s]
3154it [00:00, 96116.04it/s]
4607it [00:00, 116973.23it/s]
16550it [00:00, 129898.84it/s]
10464it [00:00, 108132.63it/s]
28238it [00:00, 121318.49it/s]
8276it [00:00, 89350.08it/s]
16958it [00:00, 123315.90it/s]
5128it [00:00, 90817.46it/s]
1974it [00:00, 93988.67it/s]
32527it [00:00, 124345.14it/s]
18672it [00:00, 101224.72it/s]
8216it [00:00, 97922.51it/s]
114650it [00:00, 128665.73it/s]
12840it [00:00, 56321.58it/s]
46206it [00:00, 127609.71it/s]
2926it [

6535it [00:00, 122576.85it/s]
9318it [00:00, 119849.38it/s]
143004it [00:01, 126170.17it/s]
32865it [00:00, 77784.19it/s]
21814it [00:00, 108412.41it/s]
8520it [00:00, 93596.13it/s]
15470it [00:00, 121202.73it/s]
5504it [00:00, 95217.36it/s]
12782it [00:00, 112764.91it/s]
158032it [00:01, 127259.88it/s]
3354it [00:00, 15831.00it/s]
5053it [00:00, 116632.83it/s]
145260it [00:01, 129176.37it/s]
77919it [00:00, 99927.04it/s]
8045it [00:00, 50368.36it/s]
12010it [00:00, 114100.60it/s]
9405it [00:00, 106087.39it/s]
16367it [00:00, 114846.40it/s]
1509it [00:00, 47450.29it/s]
13356it [00:00, 130329.77it/s]
2677it [00:00, 76256.45it/s]
5892it [00:00, 120643.42it/s]
1853it [00:00, 84186.84it/s]
5187it [00:00, 113540.01it/s]
2882it [00:00, 94026.01it/s]
44640it [00:00, 128766.08it/s]
30274it [00:00, 101016.27it/s]
21469it [00:00, 100955.11it/s]
24074it [00:00, 109231.23it/s]
105336it [00:00, 119840.67it/s]
22059it [00:00, 77764.67it/s]
660it [00:00, 22505.29it/s]
3204it [00:00, 127365.13it/s]
80

## Déplacement des nouveaux fichiers.

In [67]:
!mkdir -p lemmatized/xml
!mv lemmatized/tsv/*.xml lemmatized/xml/

# Zip pour partage des données

In [68]:
!zip data lemmatized/xml/*.xml

updating: lemmatized/xml/urn:cts:greekLit:tlg0526.tlg003.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:anthoLat.anthoLat.musisquedeoque-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:dlt000006.001.csl-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0022.phi001.thayer-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0119.phi001.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0119.phi002.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0119.phi003.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0119.phi004.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0119.phi005.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0119.phi006.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0119.phi007.perseus-lat2.xml (deflated 89%)
updating: lemmatized/

updating: lemmatized/xml/urn:cts:latinLit:phi0690.phi003.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0830.phi001.thayer-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0836.phi002.perseus-lat4.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0836.phi002.perseus-lat5.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0836.phi002.perseus-lat6.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0845.phi002.perseus-lat3.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0860.phi001.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0887.phi001.thayer-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0893.phi001.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0893.phi002.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:phi0893.phi003.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xm

updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa006.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa0071.digilibLT-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa011.opp-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa011.opp-lat3.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa011.opp-lat4.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa014a.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa015b.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa016.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa017.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa019.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa020.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:l

updating: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa011.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa012.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa013.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa014.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa015.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104c.stoa001.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104p.stoa001.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104p.stoa002.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104p.stoa003.opp-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104p.stoa005.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0104p.stoa006.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts

updating: lemmatized/xml/urn:cts:latinLit:stoa0238.stoa009.perseus-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0241d.stoa002.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0244d.stoa005.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0245a.stoa001.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0245c.stoa001.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0247.stoa001.thayer-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0249a.stoa002.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0249a.stoa003.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0249a.stoa003.opp-lat2.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0249a.stoa004.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn:cts:latinLit:stoa0251.stoa001.opp-lat1.xml (deflated 89%)
updating: lemmatized/xml/urn